In [6]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2




from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Band one
duet = config.Telescope(config='midex')
bandone = duet.bandpass1
bandtwo = duet.bandpass2
print('For 15e3 K source')
swiftmag = 18 * u.ABmag
dmag = 0.1 * u.mag
band1_caught = False
band2_caught = False
while not (band1_caught & band2_caught):
    swiftmag += dmag
    band1_fluence, band2_fluence = bb_abmag_fluence(duet = duet, swiftmag=swiftmag, bbtemp=15e3*u.K)
    
    band1_rate = duet.fluence_to_rate(band1_fluence)
    band2_rate = duet.fluence_to_rate(band2_fluence)

    if band1_caught is False:
        if (band1_rate.value < 1.0):            
            bbmag1, foo = bb_abmag(swiftmag=swiftmag, bbtemp=15e3*u.K, bandone = bandone)
            print('Band 1 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
            print('Band 1 Photometric Zero Point: {}'.format(bbmag1))
            print()
            band1_caught = True
        
    if band2_caught is False:
        if (band2_rate.value < 1.0):            
            foo, bbmag2 = bb_abmag(swiftmag=swiftmag, bbtemp=15e3*u.K, bandtwo = bandtwo)
            print('Band 2 Photometric Zero Point (Swift UVW2): {}'.format(swiftmag))
            print('Band 2 Photometric Zero Point: {}'.format(bbmag2))
            print()
            band2_caught = True

For 15e3 K source


Band 1 Photometric Zero Point (Swift UVW2): 21.40000000000005 mag(AB)
Band 1 Photometric Zero Point: 21.430542455797394 mag(AB)

Band 2 Photometric Zero Point (Swift UVW2): 21.700000000000053 mag(AB)
Band 2 Photometric Zero Point: 21.29198070324437 mag(AB)



In [5]:
# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope(config='midex')

print(duet.read_noise)

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 #+ bgd_band2

    for nframes in [1, 2, 3, 4, 5, 6, 7]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
#            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate #+ band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print(f'DUET1 Band {nframes*exposure} {siglimit}-σ magnitude limit: {bbmag1:8.2f}')

4


B


DUET1 Band 300.0 s 5-σ magnitude limit:    22.33 mag(AB)
DUET1 Band 600.0 s 5-σ magnitude limit:    22.78 mag(AB)
DUET1 Band 900.0 s 5-σ magnitude limit:    23.03 mag(AB)
DUET1 Band 1200.0 s 5-σ magnitude limit:    23.18 mag(AB)
DUET1 Band 1500.0 s 5-σ magnitude limit:    23.33 mag(AB)
DUET1 Band 1800.0 s 5-σ magnitude limit:    23.43 mag(AB)
DUET1 Band 2100.0 s 5-σ magnitude limit:    23.53 mag(AB)


In [6]:
# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope(config='midex')
print(duet.read_noise)
duet.read_noise = (2**0.5) * duet.read_noise
duet.read_noise = 0.

print(duet.read_noise)

bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 #+ bgd_band2

    for nframes in [1, 2, 3, 4, 5, 6, 7]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
#            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate #+ band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print(f'DUET1 Band {nframes*exposure} {siglimit}-σ magnitude limit: {bbmag1:8.2f}')

4
0.0


B


DUET1 Band 300.0 s 5-σ magnitude limit:    23.48 mag(AB)
DUET1 Band 600.0 s 5-σ magnitude limit:    24.18 mag(AB)
DUET1 Band 900.0 s 5-σ magnitude limit:    24.58 mag(AB)
DUET1 Band 1200.0 s 5-σ magnitude limit:    24.83 mag(AB)
DUET1 Band 1500.0 s 5-σ magnitude limit:    25.03 mag(AB)
DUET1 Band 1800.0 s 5-σ magnitude limit:    25.18 mag(AB)
DUET1 Band 2100.0 s 5-σ magnitude limit:    25.33 mag(AB)


In [4]:
# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope(config='midex')
duet.read_noise = 2.


bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 #+ bgd_band2

    for nframes in [1, 2, 3, 4, 5, 6, 7]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
#            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate #+ band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('DUET1 Band {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))



B


DUET1 Band 300.0 s 5-σ magnitude limit: 23.330542455797392 mag(AB)
DUET1 Band 600.0 s 5-σ magnitude limit: 23.830542455797406 mag(AB)
DUET1 Band 900.0 s 5-σ magnitude limit: 24.130542455797404 mag(AB)
DUET1 Band 1200.0 s 5-σ magnitude limit: 24.28054245579741 mag(AB)
DUET1 Band 1500.0 s 5-σ magnitude limit: 24.430542455797415 mag(AB)
DUET1 Band 1800.0 s 5-σ magnitude limit: 24.580542455797413 mag(AB)
DUET1 Band 2100.0 s 5-σ magnitude limit: 24.680542455797415 mag(AB)


In [8]:
duet = config.Telescope(config='midex')
info = duet.info()

-----
        DUET Telescope State: midex
        Physical Entrance Pupil: 50.0 cm
        Effective EPD: 40.0 cm
        Effective Area: 1256.6370614359173 cm2
        
        Pixel size: 1.0 arcsec
        Pointing RMS: 2.5 arcsec
        Effective PSF FWHM: 2.0 arcsec
        N_eff: 10.16620230134607

        Band 1: {'eff_wave': <Quantity 198.63858525 nm>, 'eff_width': <Quantity 31.00790249 nm>}
        Bandpass 1: [183.13463401 214.1425365 ] nm
        Band 2: {'eff_wave': <Quantity 280.34471808 nm>, 'eff_width': <Quantity 41.64586798 nm>}
        Bandpass 2: [259.5217841  301.16765207] nm

        Dark current: 0.0014 ph / s
        Read noise (RMS per read): 4
        -----
        


In [9]:
# Midex size:
side = ((1024) * 12 * duet.pixel).to(u.deg).value
fov = side**2
print(f'FoV {fov:8.2f} deg2')

FoV    11.65 deg2


In [10]:
sky_survey = ( 4 * np.pi * (180/np.pi)**2)
print(f'Sky Area: {sky_survey:8.2f}')
tiles = 1.*(sky_survey / fov)
print(f'Number of tiles (50% buffer): {tiles:8.2f}')


tiles = 1.5*(sky_survey / fov)
print(f'Number of tiles (50% buffer): {tiles:8.2f}')

duration = 1.2*(tiles * 300 * u.s).to(u.d)
print(f'Days to complete: {duration:8.2f}')



Sky Area: 41252.96
Number of tiles (50% buffer):  3540.77
Number of tiles (50% buffer):  5311.16
Days to complete:    22.13 d


In [18]:
n = 1.4
d = 500 # mm
f = d * n

print(f) # 650 mm
p = 206265 / f
print(p * 1e-3)




700.0
0.2946642857142857


In [23]:
p = 1 # 1 arcsec per pixel
d = 500 # mm
f = 206265 / p # mm
print(f)
n = f / d
print(n)



206265.0
412.53
